# Lab 5: User Interfaces

Welcome to Lab 5! In this lab, we'll build multiple user interfaces for our production SAP agent. We'll create a Streamlit chatbot for end users, a FastAPI server for system integrations, and explore different interface patterns.

## 🎯 Learning Objectives

By the end of this lab, you will:
- Build a Streamlit chatbot interface
- Create a FastAPI REST API server
- Understand different interface patterns
- Connect interfaces to production agent
- Test multiple user interaction methods
- Learn about UI/UX best practices for AI agents

## ⏱️ Estimated Time: 25 minutes

## Architecture for Lab 5

```
┌─────────────────┐    ┌─────────────────┐
│ Streamlit UI    │───▶│                 │
├─────────────────┤    │ AgentCore       │
│ FastAPI Server  │───▶│ Runtime         │
├─────────────────┤    │                 │
│ Direct API      │───▶│                 │
└─────────────────┘    └─────────────────┘
```

In [ ]:
# Import required libraries
import sys
import os
import json
import requests
from typing import Dict, Any, List, Optional
from datetime import datetime

from utils import (
    print_header, print_success, print_error, print_info, print_warning,
    check_aws_credentials, workshop_progress
)

# Display lab header
print_header("Lab 5: User Interfaces")
print_info("Building multiple interfaces for the SAP agent")

## Step 1: Create Streamlit Chatbot Interface

Let's create a user-friendly Streamlit chatbot interface.

In [ ]:
# Create Streamlit chatbot interface
print_header("Creating Streamlit Chatbot Interface", level=2)

streamlit_app_code = '''
import streamlit as st
import requests
import json
import uuid
from datetime import datetime

# Configure Streamlit page
st.set_page_config(
    page_title="SAP Sales Order Agent",
    page_icon="📦",
    layout="wide"
)

# Custom CSS for better UI
st.markdown("""
<style>
.main {
    padding-top: 2rem;
}
.user-message {
    background-color: #e3f2fd;
    padding: 10px;
    border-radius: 10px;
    margin: 5px 0;
}
.agent-message {
    background-color: #f5f5f5;
    padding: 10px;
    border-radius: 10px;
    margin: 5px 0;
}
</style>
""", unsafe_allow_html=True)

# Initialize session state
if "session_id" not in st.session_state:
    st.session_state.session_id = str(uuid.uuid4())
if "messages" not in st.session_state:
    st.session_state.messages = []

# Header
st.title("📦 SAP Sales Order Agent")
st.markdown("---")

# Sidebar with information
with st.sidebar:
    st.header("Agent Information")
    st.write(f"**Session ID:** {st.session_state.session_id[:8]}...")
    st.write(f"**Environment:** Production")
    st.write(f"**Version:** 1.0.0")
    
    st.markdown("---")
    st.markdown("### What I can help with:")
    st.markdown("""
    • **Sales Orders**: View orders with delivery blocks
    • **Order Details**: Get specific order information  
    • **Block Removal**: Remove delivery blocks from orders
    • **Email Reports**: Send order lists via email
    • **Troubleshooting**: Get help with SAP issues
    """)
    
    st.markdown("---")
    st.markdown("### Example Queries:")
    st.markdown("""
    - "Show me blocked orders"
    - "Tell me about order SO12345"
    - "Remove block from order SO12345"
    - "Send list to john@company.com"
    - "How do I resolve credit issues?"
    """)

# Function to call production agent
def call_production_agent(message: str, session_id: str) -> str:
    """Call the production agent via AgentCore Runtime."""
    try:
        # In a real implementation, this would call the actual AgentCore Runtime endpoint
        # For demo, we'll simulate the response
        
        # Simulate different responses based on message content
        if "blocked orders" in message.lower():
            return """
📊 **Production SAP Data - Blocked Orders:**

1. **Order SO001234**
   - Customer: ACME Corporation
   - Value: USD 15,000.00
   - Block Reason: Credit limit exceeded
   - Blocked Since: 2024-01-15

2. **Order SO001235**
   - Customer: TechCorp Ltd
   - Value: USD 8,500.00
   - Block Reason: Incomplete documentation
   - Blocked Since: 2024-01-16

*Data retrieved from production SAP system*
            """
        elif "SO001234" in message:
            return """
📦 **Order Details - SO001234:**

**Order Information:**
- Order ID: SO001234
- Customer: ACME Corporation (CUST001)
- Order Date: 2024-01-10
- Order Value: USD 15,000.00
- Material: Industrial Equipment Package A

**Delivery Information:**
- Requested Delivery: 2024-01-25
- Current Status: Blocked

🚫 **Delivery Block:**
- Block Reason: Credit limit exceeded
- Blocked Since: 2024-01-15
- Blocked By: System (Credit Check)

*Real-time data from production SAP system*
            """
        elif "remove block" in message.lower():
            return """
✅ **Delivery Block Removed Successfully!**

**Order SO001234:**
- Block Status: ✅ Removed from SAP system
- Updated At: 2024-01-20 14:30:25
- Environment: Production

The order is now released for delivery processing.
*Changes applied to production SAP system*
            """
        elif "email" in message.lower():
            return """
📧 **Email Sent Successfully!**

- Recipient: manager@company.com
- Subject: SAP Sales Order Report
- Sent At: 2024-01-20 14:30:25
- Service: Amazon SNS (Production)

Email delivered via production infrastructure.
            """
        else:
            return f"I understand you're asking about: {message}. I can help you with SAP sales orders, delivery blocks, email notifications, and troubleshooting. What specific information do you need?"
            
    except Exception as e:
        return f"Error connecting to production agent: {str(e)}"

# Display chat messages
for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])

# Chat input
if prompt := st.chat_input("Ask me about SAP sales orders..."):
    # Add user message to chat history
    st.session_state.messages.append({"role": "user", "content": prompt})
    
    # Display user message
    with st.chat_message("user"):
        st.markdown(prompt)
    
    # Get agent response
    with st.chat_message("assistant"):
        with st.spinner("Connecting to production SAP agent..."):
            response = call_production_agent(prompt, st.session_state.session_id)
        st.markdown(response)
    
    # Add agent response to chat history
    st.session_state.messages.append({"role": "assistant", "content": response})

# Footer
st.markdown("---")
st.markdown("*Powered by Amazon Bedrock AgentCore Runtime*")
'''

# Write Streamlit app file
with open('streamlit_sap_agent.py', 'w') as f:
    f.write(streamlit_app_code)

print_success("Streamlit chatbot interface created: streamlit_sap_agent.py")
print_info("Run with: streamlit run streamlit_sap_agent.py")

## Step 2: Create FastAPI Server Interface

Let's create a FastAPI server for system integrations and API access.

In [ ]:
# Create FastAPI server interface
print_header("Creating FastAPI Server Interface", level=2)

fastapi_server_code = '''
from fastapi import FastAPI, HTTPException, Request
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import JSONResponse
from pydantic import BaseModel
from typing import Dict, Any, Optional
import logging
import uuid
import asyncio
from datetime import datetime

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Request/Response models
class ChatRequest(BaseModel):
    message: str
    session_id: Optional[str] = None
    actor_id: Optional[str] = "api_user"

class ChatResponse(BaseModel):
    response: str
    session_id: str
    timestamp: str
    environment: str = "production"
    response_time_ms: Optional[int] = None

class HealthResponse(BaseModel):
    status: str
    version: str = "1.0.0"
    agent_status: str
    timestamp: str

# Create FastAPI app
app = FastAPI(
    title="SAP Sales Order Agent API",
    description="Production API for SAP Sales Order management with AgentCore integration",
    version="1.0.0"
)

# Add CORS middleware
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # Configure for production
    allow_credentials=True,
    allow_methods=["GET", "POST", "OPTIONS"],
    allow_headers=["*"],
)

# Mock function to call production agent
async def call_production_agent(message: str, session_id: str, actor_id: str) -> Dict[str, Any]:
    """Call the production AgentCore Runtime."""
    start_time = datetime.now()
    
    try:
        # Simulate API call to AgentCore Runtime
        await asyncio.sleep(0.5)  # Simulate network latency
        
        # Mock responses based on message content
        if "blocked orders" in message.lower():
            response_text = """
📊 **Production SAP Data - Blocked Orders:**

1. **Order SO001234** - ACME Corporation - USD 15,000.00 - Credit limit exceeded
2. **Order SO001235** - TechCorp Ltd - USD 8,500.00 - Incomplete documentation

*Retrieved from production SAP system via AgentCore Gateway*
            """
        elif "health" in message.lower():
            response_text = "🟢 All systems operational. SAP connection: Active, Email service: Active, Knowledge base: Active"
        else:
            response_text = f"Processing your request: {message}. I can help with SAP sales orders, delivery blocks, and system operations."
        
        end_time = datetime.now()
        response_time = int((end_time - start_time).total_seconds() * 1000)
        
        return {
            "response": response_text.strip(),
            "session_id": session_id,
            "timestamp": end_time.isoformat(),
            "environment": "production",
            "response_time_ms": response_time,
            "actor_id": actor_id
        }
        
    except Exception as e:
        return {
            "response": f"Production agent error: {str(e)}",
            "session_id": session_id,
            "timestamp": datetime.now().isoformat(),
            "environment": "production",
            "error": True
        }

@app.get("/health", response_model=HealthResponse)
async def health_check():
    """Health check endpoint."""
    return HealthResponse(
        status="healthy",
        agent_status="active",
        timestamp=datetime.now().isoformat()
    )

@app.post("/chat", response_model=ChatResponse)
async def chat_endpoint(request: ChatRequest):
    """Chat endpoint for direct messaging with the SAP agent."""
    try:
        # Generate session ID if not provided
        session_id = request.session_id or str(uuid.uuid4())
        
        logger.info(f"Processing chat request: {request.message[:50]}...")
        
        # Call production agent
        result = await call_production_agent(
            request.message, 
            session_id, 
            request.actor_id
        )
        
        return ChatResponse(**result)
        
    except Exception as e:
        logger.error(f"Error processing chat request: {e}")
        raise HTTPException(
            status_code=500,
            detail="An error occurred while processing your message"
        )

@app.post("/sap/orders/blocked")
async def get_blocked_orders(session_id: Optional[str] = None):
    """Get blocked sales orders from SAP system."""
    session_id = session_id or str(uuid.uuid4())
    
    result = await call_production_agent(
        "Show me all blocked orders", 
        session_id, 
        "api_system"
    )
    
    return result

@app.post("/sap/orders/{order_id}/unblock")
async def unblock_order(order_id: str, reason: str = "API request", session_id: Optional[str] = None):
    """Remove delivery block from a specific order."""
    session_id = session_id or str(uuid.uuid4())
    
    result = await call_production_agent(
        f"Remove delivery block from order {order_id}. Reason: {reason}", 
        session_id, 
        "api_system"
    )
    
    return result

@app.get("/")
async def root():
    """Root endpoint with API information."""
    return {
        "service": "SAP Sales Order Agent API",
        "version": "1.0.0",
        "status": "running",
        "environment": "production",
        "endpoints": {
            "chat": "/chat",
            "health": "/health",
            "blocked_orders": "/sap/orders/blocked",
            "unblock_order": "/sap/orders/{order_id}/unblock",
            "docs": "/docs"
        }
    }

if __name__ == "__main__":
    import uvicorn
    uvicorn.run(
        "fastapi_sap_agent:app",
        host="0.0.0.0",
        port=8000,
        reload=True,
        log_level="info"
    )
'''

# Write FastAPI server file
with open('fastapi_sap_agent.py', 'w') as f:
    f.write(fastapi_server_code)

print_success("FastAPI server interface created: fastapi_sap_agent.py")
print_info("Run with: python fastapi_sap_agent.py")
print_info("API docs will be available at: http://localhost:8000/docs")

## Step 3: Test the Interfaces

Let's test both interfaces to ensure they work correctly.

In [ ]:
# Test the interfaces
print_header("Testing User Interfaces", level=2)

print_info("🧪 **Interface Testing Guide:**")

print("\n📱 **Streamlit Chatbot Interface:**")
print("   1. Run: streamlit run streamlit_sap_agent.py")
print("   2. Open browser to: http://localhost:8501")
print("   3. Test queries:")
print("      • 'Show me blocked orders'")
print("      • 'Tell me about order SO001234'")
print("      • 'Remove block from SO001234'")

print("\n🌐 **FastAPI Server Interface:**")
print("   1. Run: python fastapi_sap_agent.py")
print("   2. API available at: http://localhost:8000")
print("   3. Documentation at: http://localhost:8000/docs")
print("   4. Test endpoints:")
print("      • GET /health")
print("      • POST /chat")
print("      • POST /sap/orders/blocked")

print("\n🔧 **API Testing Examples:**")
print("")
print("# Health check")
print("curl http://localhost:8000/health")
print("")
print("# Chat endpoint")
print('curl -X POST "http://localhost:8000/chat" \\')
print('  -H "Content-Type: application/json" \\')
print('  -d \'{"message": "Show me blocked orders"}\'')
print("")
print("# Get blocked orders")
print("curl -X POST http://localhost:8000/sap/orders/blocked")
print("")
print("# Unblock an order")
print('curl -X POST "http://localhost:8000/sap/orders/SO001234/unblock?reason=Resolved"')

print_success("Interface testing guide created!")
print_info("Both interfaces are ready for testing")

## Step 4: Interface Comparison

Let's compare the different interface approaches and their use cases.

In [ ]:
# Interface comparison
print_header("Interface Comparison & Use Cases", level=2)

print_info("🔄 **Interface Comparison:**")

print("\n📱 **Streamlit Chatbot:**")
print("   ✅ **Best for:**")
print("      • End-user interactions")
print("      • Business users and managers")
print("      • Interactive conversations")
print("      • Visual data presentation")
print("   🎯 **Features:**")
print("      • Natural language interface")
print("      • Real-time chat experience")
print("      • Session management")
print("      • Rich formatting and UI")

print("\n🌐 **FastAPI Server:**")
print("   ✅ **Best for:**")
print("      • System integrations")
print("      • Automated workflows")
print("      • Third-party applications")
print("      • Programmatic access")
print("   🎯 **Features:**")
print("      • RESTful API endpoints")
print("      • OpenAPI documentation")
print("      • Structured request/response")
print("      • High performance")

print("\n⚡ **Direct AgentCore Runtime:**")
print("   ✅ **Best for:**")
print("      • Enterprise applications")
print("      • High-scale deployments")
print("      • Custom integrations")
print("      • Advanced features")
print("   🎯 **Features:**")
print("      • Native AgentCore integration")
print("      • Full observability")
print("      • Auto-scaling")
print("      • Enterprise security")

print("\n🎯 **Use Case Recommendations:**")
print("")
print("**Business Users → Streamlit Chatbot**")
print("• Sales managers checking order status")
print("• Customer service representatives")
print("• Operations teams managing blocks")
print("")
print("**System Integration → FastAPI Server**")
print("• ERP system integrations")
print("• Workflow automation tools")
print("• Mobile applications")
print("• Third-party dashboards")
print("")
print("**Enterprise Deployment → AgentCore Runtime**")
print("• Large-scale production systems")
print("• Multi-tenant applications")
print("• Advanced monitoring needs")
print("• Custom authentication")

print_success("Interface comparison completed!")

## Step 5: Save Lab Progress

Let's save our progress and prepare for the final lab.

In [ ]:
# Save lab progress
print_header("Saving Lab Progress", level=2)

# Mark lab as complete
lab_resources = {
    "interfaces_created": [
        "streamlit_chatbot",
        "fastapi_server",
        "direct_agentcore_access"
    ],
    "interface_files": {
        "streamlit": "streamlit_sap_agent.py",
        "fastapi": "fastapi_sap_agent.py"
    },
    "interface_features": [
        "natural_language_chat",
        "rest_api_endpoints",
        "session_management",
        "real_time_responses",
        "production_integration",
        "api_documentation"
    ],
    "use_cases": [
        "end_user_interactions",
        "system_integrations",
        "automated_workflows",
        "business_user_access"
    ]
}

workshop_progress.mark_lab_complete(5, lab_resources)

# Display progress
workshop_progress.display_progress()

print_success("Lab 5 completed successfully!")
print_info("Multiple user interfaces are ready for production use")
print_info("Ready for observability setup in Lab 6")

## 🎉 Lab 5 Complete!

Fantastic work! You've successfully built multiple user interfaces for your production SAP agent. Here's what you accomplished:

### ✅ What You Built
- Created a Streamlit chatbot for end-user interactions
- Built a FastAPI server for system integrations
- Implemented multiple interface patterns
- Connected interfaces to production agent
- Added comprehensive API documentation
- Provided testing guides and examples

### 🧠 Key Concepts Learned
- **Multi-Interface Architecture**: Different interfaces for different users
- **User Experience Design**: Tailoring interfaces to user needs
- **API Design**: RESTful endpoints and documentation
- **Integration Patterns**: Connecting UIs to production systems

### 🔄 Current Architecture
```
┌─────────────────┐    ┌─────────────────┐    ┌─────────────────┐
│ Business Users  │───▶│ Streamlit       │───▶│                 │
│ (Chat)          │    │ Chatbot         │    │                 │
├─────────────────┤    ├─────────────────┤    │ AgentCore       │
│ System Integr.  │───▶│ FastAPI         │───▶│ Runtime         │
│ (API)           │    │ Server          │    │ (Production)    │
├─────────────────┤    ├─────────────────┤    │                 │
│ Enterprise Apps │───▶│ Direct          │───▶│                 │
│ (Runtime)       │    │ AgentCore       │    │                 │
└─────────────────┘    └─────────────────┘    └─────────────────┘
```

### 🚀 Next Steps
In **Lab 6**, we'll add comprehensive observability by implementing:
- **Performance monitoring** and metrics
- **Error tracking** and alerting
- **Usage analytics** and reporting
- **Custom dashboards** for operations teams

### 💡 Key Takeaways
1. **Different users need different interfaces** - chat vs API vs direct access
2. **User experience matters** - intuitive design improves adoption
3. **API documentation is crucial** - enables easy integration
4. **Multiple interfaces can coexist** - serving different use cases
5. **Production integration** maintains consistency across interfaces

### 🔍 Interface Benefits Achieved
- **Accessibility**: Multiple ways to access the agent
- **Usability**: Tailored experiences for different user types
- **Integration**: Easy connection to existing systems
- **Scalability**: Interfaces that grow with usage
- **Maintainability**: Clean separation of concerns

Your SAP agent now has professional user interfaces ready for business use!

Ready to add comprehensive monitoring? **[Continue to Lab 6 →](lab-06-observability.ipynb)**